# <p style="background-color:lightgreen;font-family:newtimeroman;color:black;font-size:150%;text-align:center;border-radius:50px 50px;">Введение в обработку естественного языка</p>

<h2 align='center'>Урок 3. Embedding word2vec fasttext</h2>  

<h3 align='left'>Практическое задание:</h3>  

Скачиваем датасет

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

Или данные можно скачать тут https://github.com/IlyaGusev/gazeta

Пример работы с ним:

from corus import load_ods_gazeta  
path = 'gazeta.csv.gz'  
records = load_ods_gazeta(path)  
next(records)  

что надо сделать:

на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
Проверить насколько хорошо работают подходы

### Библиотеки

In [1]:
# !pip -qq install pymorphy2
# !pip -qq install stop_words
# !pip install gensim

In [2]:
import numpy as np

import tarfile
import json

from tqdm import tqdm

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

from gensim.models import Word2Vec, FastText
from sklearn.neighbors import NearestNeighbors

### Данные:

In [3]:
data = []

tar = tarfile.open("./data/gazeta_jsonl.tar.gz", "r:gz")
for member in tar.getmembers():
     f = tar.extractfile(member)
     if f is not None:
         content = f.read().decode('utf-8')
         content = content.split('\n')
         # последний элемент пустой, так как строка оканчивалась на \n
         content = content[:-1]
         content = [json.loads(dic) for dic in content]
         data.extend(content)

In [4]:
len(data)

63435

In [5]:
data[1].keys()

dict_keys(['url', 'text', 'title', 'summary', 'date'])

In [6]:
print(f" {data[1]['url']}\n {data[1]['title']}\n {data[1]['summary']}")

 https://www.gazeta.ru/business/2013/01/24/4939629.shtml
 Google закончил поиск
 Юлия Соловьева, экс-директор холдинга «Профмедиа», возглавила российский офис Google. Поиск американская интернет-корпорация вела полгода. Медиаменеджер с опытом работы в международных компаниях найдет общий язык с акционерами, но прорыва можно было ждать от технического специалиста или маркетолога, говорят аналитики.


In [7]:
data[1]['text'][:100]

'Российское подразделение интернет-корпорации Google возглавила Юлия Соловьева , экс-директор московс'

In [8]:
data[1]['date']

'2013-01-24 18:20:09'

### Обработаем данные

In [9]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [10]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0


for line in tqdm(data):
    spls = preprocess_txt(line['summary'])
    sentences.append(spls)
    c += 1
    if c > 100000:
        break

100%|████████████████████████████████████████████████████████████████████████| 63435/63435 [06:42<00:00, 157.65it/s]


In [11]:
sentences = [i for i in sentences if len(i) > 2]

In [12]:
sentences[1][:10]

['юлия',
 'соловьёв',
 'эксдиректор',
 'холдинг',
 '«профмедиа»',
 'возглавить',
 'российский',
 'офис',
 'google',
 'поиск']

### Обучим модели

In [13]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)

In [14]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [16]:
modelW2V.save('./models/modelW2V')

In [17]:
modelFT.save('./models/modelFT')

In [19]:
modelW2V = Word2Vec.load('./models/modelW2V')

In [20]:
modelFT = FastText.load('./models/modelFT')

### Составим векторы

In [26]:
W2V_sentences_vectors = []
FT_sentences_vectors = []

for sentence in sentences:
    W2V_vector = np.zeros(300)
    W2V_c = 0
    FT_vector = np.zeros(300)
    for word in sentence:
        if word in modelW2V.wv:
            W2V_vector += modelW2V.wv[word]  # get numpy vector of a word
            W2V_c += 1
        FT_vector += modelFT.wv[word]
    if W2V_c>0:
        W2V_vector = W2V_vector/W2V_c  # нормализация
    W2V_sentences_vectors.append(W2V_vector)
    FT_vector = FT_vector/len(sentence)
    FT_sentences_vectors.append(FT_vector)

In [30]:
W2V_sentences_vectors[1][:10]

array([-0.22351677,  0.30044088,  0.19219234,  0.27692538, -0.30145015,
       -0.44695619,  0.17514304,  0.72611031, -0.25467916, -0.13125373])

In [31]:
FT_sentences_vectors[1][:10]

array([-0.01668978,  0.07741924, -0.11366574, -0.00528717,  0.04816609,
       -0.1355478 ,  0.4453918 ,  0.1419937 , -0.09991112, -0.3055655 ])

### Поиск

In [49]:
N_NEIGHBORS = 5 

In [50]:
W2V_neighbour = NearestNeighbors(n_neighbors=N_NEIGHBORS)
W2V_neighbour.fit(W2V_sentences_vectors)
FT_neighbour = NearestNeighbors(n_neighbors=N_NEIGHBORS)
FT_neighbour.fit(FT_sentences_vectors)

NearestNeighbors()

In [87]:
def get_response(question, model, neighbourhood):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = neighbourhood.kneighbors([vector], return_distance=False)
    return [data[i] for i in answers[0]]   

Выведим краткое содержание для сравнения, а так можно получить доступ и ко всему тексту, если указать 'text' вместо 'summary'

Погода

In [100]:
TEXT1 = 'Погода на завтра'
FULL = 'summary' # text - полное содержание новости, summary - заголовок

In [101]:
%%time

result = get_response(TEXT1, modelW2V, W2V_neighbour)
for i in range(len(result)):
  print(f'{i+1}.', result[i][FULL])

1. До выходных в столичном регионе будет сохраняться аномально теплая погода. О плюсовой температуре, дожде и сильном ветре москвичи смогут забыть только в середине следующей недели, когда столбики термометров поползут вниз и выпадет снег. Тогда же, по прогнозам синоптиков, погода вернется к климатической норме, то есть легкому морозцу.
2. Ровно через неделю москвичей и гостей столицы ожидает 20-градусное тепло. А уже в эту пятницу температура приблизится к климатической норме. Как прогнозируют метеорологи, температура воздуха в течение недели будет повышаться постепенно, при этом погода будет иметь неустойчивый характер.
3. Столичный регион в выходные и на следующей неделе попеременно будут проходить холодные и теплые атмосферные фронты. Температура в выходные будет немного выше нормы. В будни воздух будет прогреваться не так сильно, но погода будет стоять сухая и ясная.
4. На протяжении всей недели температура будет понижаться. К следующим выходным кратковременные дожди сменят заморо

In [102]:
%%time

result = get_response(TEXT1, modelFT, FT_neighbour)
for i in range(len(result)):
  print(f'{i+1}.', result[i][FULL])

1. Погода в Москве и Подмосковье немного ухудшится: в выходные будет хоть и тепло, но местами пройдут дожди. На следующей неделе погода также будет неустойчивой, а ближе к концу апреля прогремят грозы. Температура, впрочем, порадует: она будет на несколько градусов выше нормы.
2. В ближайшие пятницу и выходные в столице ожидается выпадение почти трети месячной нормы осадков, или, как сформулировали синоптики, по два ведра дождевой воды на квадратный метр поверхности. Впрочем, к концу следующей недели москвичам обещают несколько сухих и не по сезону теплых дней.
3. До выходных в столичном регионе будет сохраняться аномально теплая погода. О плюсовой температуре, дожде и сильном ветре москвичи смогут забыть только в середине следующей недели, когда столбики термометров поползут вниз и выпадет снег. Тогда же, по прогнозам синоптиков, погода вернется к климатической норме, то есть легкому морозцу.
4. Метеорологи предупредили москвичей об ухудшении погоды в начале зимы. После краткосрочного

Общество

In [105]:
TEXT2 = 'Жизнь замечательных людей'
FULL = 'summary' # text - полное содержание новости, summary - заголовок

In [106]:
%%time

result = get_response(TEXT2, modelW2V, W2V_neighbour)
for i in range(len(result)):
  print(f'{i+1}.', result[i][FULL])

1. Вчера опять прибежала какая-то однокурсница, которую я видел пару раз, да и то мельком, и слёзно умоляла «спасти его». И я возвращал ей ее «его» к жизни до четырёх утра (!!!).
2. Дневник ММКФ: Годар и Трюффо работают кинокритиками, сентиментальные мадьярки тоскуют по родине, Шаброль снова приглашает в «Ад».
3. Термин «офисный компьютер» сегодня всё чаще означает именно «ноутбук», а не более привычный конторским труженикам со стажем светло-серый «мини-тауэр» с замызганными клавиатурой, мышкой и подслеповатым монитором на 17 дюймов.
4. Ренуар и Суриков, Моне и Репин, Писсаро и Коровин, Пикассо и Серов, Менье и Архипов — в ГМИИ им.А.С. Пушкина открылась выставка-квест «Передвижники и импрессионисты. На пути в XX век».
5. Концерт Ника Кейва с проектом Grinderman в полупустом «Крокус Сити Холле» превратился в инфернальную мистерию с надеванием заячьих ушек, похабными блюзами и красноволосыми танцовщицами.
CPU times: user 475 ms, sys: 0 ns, total: 475 ms
Wall time: 73 ms


In [107]:
%%time

result = get_response(TEXT2, modelFT, FT_neighbour)
for i in range(len(result)):
  print(f'{i+1}.', result[i][FULL])

1. В России могут ввести новый обязательный школьный курс «Нравственные основы семейной жизни». В министерстве образования и науки эту идею обещали рассмотреть и оценить. Современная школа в последние годы стала чуть ли не главной экспериментальной площадкой поиска некоей новой национальной идеологии. Вот только раз за разом эта идеология сводится к желанию залезть в личную жизнь граждан – как взрослых, так и детей.
2. Наш общественный строй исключает получение школьниками современного, равнодоступного и уж тем более бесплатного образования.
3. Моральные стимулы неплохо оправдывают себя в тех социальных системах, где есть мораль и моральные авторитеты. Но возможна ли в принципе новая эффективная система моральных поощрений в стране, где люди любого осмысленного и честного труда с нищенской зарплатой являются в бытовом сознании не столько героями, сколько «лузерами». Причем как для власти, так и для молодежи, которую, по идее, с помощью этих самых моральных поощрений и надо воспитывать 

Армия

In [108]:
TEXT3 = 'Современный солдат'
FULL = 'summary' # text - полное содержание новости, summary - заголовок

In [109]:
%%time

result = get_response(TEXT3, modelW2V, W2V_neighbour)
for i in range(len(result)):
  print(f'{i+1}.', result[i][FULL])

1. В музее современного искусства PERMM открылась выставка «Лицо невесты», представляющая современное искусство Казахстана, популярное на Западе, неизвестное в России и совершенно не стремящееся соответствовать европейским стандартам contemporary art.
2. В Музее современного искусства «Гараж» стартовала триеннале современного искусства — масштабный смотр произведений художников из российских регионов, который должен разрушить монополию Москвы на современное искусство.
3. В шоколадном цеху на «Красном Октябре» открылась выставка лауреатов World Press Photo-2013. В объективе авторов гражданские войны, вооруженные конфликты и социальные катастрофы третьего мира.
4. Картина Малевича, испорченная Александром Бренером, фильм об угнанных самолетах и виртуальное государство словенских художников: в Музее современного искусства «Гараж» проходит выставка The New International – выборка искусства 1990-х.
5. Сильнее, чем фотография: в Третьяковской галерее открылась выставка «Гиперреализм. Когда р

In [110]:
%%time

result = get_response(TEXT3, modelFT, FT_neighbour)
for i in range(len(result)):
  print(f'{i+1}.', result[i][FULL])

1. Редкие снимки советских полководцев Великой Отечественной войны опубликовало российское военное ведомство. Фотографы запечатлели победителей нацизма во время торжественных парадов в Москве и других городах СССР. Многие фотографии сопровождены выдержками из воспоминаний военных и комментариями их родственников.
2. Копилка идей по реформированию Российской армии обогатилась новым экзотическим предложением. Вузы должны воспитывать не только квалифицированных физиков и лириков, но и боевых офицеров, отправляя студентов-мужчин на летний семестр на военные полигоны. Как ни странно, авторство идеи принадлежит не военным, а ученым.
3. Дмитрий Хворостовский представит в Москве новую программу «Песни военных лет – 2». Кроме знакомой всем песенной классики военных лет он исполнит русские народные песни, а также композицию из репертуара группы «Любэ».
4. 100 лет назад приказом Льва Троцкого в Рязани были созданы Пехотные курсы командного состава Красной армии, позднее преобразованные в известно

Спорт

In [113]:
TEXT4 = 'Футбольный клуб Спартак'
FULL = 'summary' # text - полное содержание новости, summary - заголовок

In [114]:
%%time

result = get_response(TEXT4, modelW2V, W2V_neighbour)
for i in range(len(result)):
  print(f'{i+1}.', result[i][FULL])

1. Главный тренер ПСЖ Карло Анчелотти хочет переманить в парижскую команду полузащитника «Челси» Фрэнка Лэмпарда. Контракт 33-летнего футболиста с лондонским клубом заканчивается в 2013 году.
2. Владелец «Челси» Роман Абрамович в свете последних не слишком удачных матчей клуба провел беседу с главным тренером команды Андре Виллаш-Боашем, в которой потребовал улучшить результаты клуба.
3. Форвард ЦСКА Андрей Воронцевич останется в клубе в следующем году. Тем не менее, агент игрока поведал, что баскетболист может уехать в НБА через год.
4. Нападающий мадридского «Реала» Криштиану Роналду дисквалифицирован Королевской испанской футбольной федерацией на пять матчей за толчок в спину главного арбитра встречи в рамках Суперкубка Испании между столичным клубом и «Барселоной».
5. Минское «Динамо» опровергло информацию о том, что клуб покинут новички команды и главный тренер Марек Сикора. Екатеринбургский «Автомобилист» допущен к участию в КХЛ в следующем сезоне, а в лигу поступают угрозы.
CPU 

In [115]:
%%time

result = get_response(TEXT4, modelFT, FT_neighbour)
for i in range(len(result)):
  print(f'{i+1}.', result[i][FULL])

1. Главный тренер ЦСКА Юлиус Шуплер извинился перед армейскими болельщиками за игру его команды в матче против СКА. Наставник клуба из Санкт-Петербурга Милош Ржига, несмотря на победу, остался недоволен игрой своих подопечных в этом матче.
2. Владелец московского «Спартака» Леонид Федун остался недоволен игрой команды в матче против «Кубани».
3. Главный тренер московского «Спартака» Массимо Каррера после разгрома испанской «Севильи» в матче Лиги чемпионов отметил, что команда провела идеальный матч против сильного соперника, а владелец красно-белых Леонид Федун пообещал, что мяч с этой игры будет выставлен в музее «Спартака».
4. Главный тренер «Зенита» Лучано Спаллетти после матча с «Твенте» сказал, что эту игру питерская команда посвятила Вячеславу Малафееву. Рулевой голландского клуба Мишель Прюдомм заметил, что после побед над «Рубином» и «Зенитом» не против получить в соперники «Спартак».
5. Главный тренер СКА Милош Ржига после поражения в третьем матче полуфинальной серии Кубка Га

Сообщение с очепятками

In [116]:
TEXT5 = 'Футбальный клуп Спыртак'
FULL = 'summary' # text - полное содержание новости, summary - заголовок

In [117]:
%%time

result = get_response(TEXT5, modelW2V, W2V_neighbour)
for i in range(len(result)):
  print(f'{i+1}.', result[i][FULL])

1. Вчера опять прибежала какая-то однокурсница, которую я видел пару раз, да и то мельком, и слёзно умоляла «спасти его». И я возвращал ей ее «его» к жизни до четырёх утра (!!!).
2. Термин «офисный компьютер» сегодня всё чаще означает именно «ноутбук», а не более привычный конторским труженикам со стажем светло-серый «мини-тауэр» с замызганными клавиатурой, мышкой и подслеповатым монитором на 17 дюймов.
3. Дневник ММКФ: Годар и Трюффо работают кинокритиками, сентиментальные мадьярки тоскуют по родине, Шаброль снова приглашает в «Ад».
4. Ренуар и Суриков, Моне и Репин, Писсаро и Коровин, Пикассо и Серов, Менье и Архипов — в ГМИИ им.А.С. Пушкина открылась выставка-квест «Передвижники и импрессионисты. На пути в XX век».
5. Концерт Ника Кейва с проектом Grinderman в полупустом «Крокус Сити Холле» превратился в инфернальную мистерию с надеванием заячьих ушек, похабными блюзами и красноволосыми танцовщицами.
CPU times: user 382 ms, sys: 3.96 ms, total: 386 ms
Wall time: 60.9 ms


In [118]:
%%time

result = get_response(TEXT5, modelFT, FT_neighbour)
for i in range(len(result)):
  print(f'{i+1}.', result[i][FULL])

1. На своем аншлаговом московском концерте Roxette продемонстрировали отличную музыкальную форму, сыграли «Калинку-малинку», а зрители радовались встрече с кумирами, как любимой футбольной команде.
2. Профессиональный хоккейный клуб ЦСКА раскрыл поставщиков одежды для команды – костюмов и других предметов гардероба от бренда Duca Sartoria, принадлежащего итальянскому дизайнеру Роберто Джиромбелли. Внешнему виду спортсменов по всему миру уделяют большое значение, ведь они представляют страну не только во время соревнований, но и на официальных мероприятиях.
3. В опубликованном Центром футбольных исследований (CIES Football Observatory) списке ста лучших футбольных академий Европы оказались сразу две представительницы Российской футбольной премьер-лиги. Московский «Спартак» занял 47-е место, а другой столичный клуб, «Локомотив», разместился на 97-й строчке.
4. В последнее время и футбольный, и хоккейный «Спартак» регулярно огорчают своих болельщиков. Остается радоваться лишь успехам моло

### Вывод:
На маой взгляд FT работает лучше. Предлагаемые им ответы точнее отражают задаваемый вопрос. Даже в варианте с опечатками, где W2V совсем ничего не показал, FT постарался сделать хоть что-то. Хотя FT работает немного медленее W2V, но результат это оправдывает.